In [1]:
# pip install networkx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\norat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
with open("./SW ep1.txt", "r") as file:
    text = file.read()

text

'STAR WARS : EPISODE 1:THE PHANTOM MENACE\n\n\n\nTITLE CARD : A long time ago in a galaxy far, far away....\n\nA vast sea of stars serves as the backdrop for the main title, followed by\na roll up, which crawls up into infinity.\n\nEPISODE 1 THE PHANTOM MENACE\n\nTurmoil has engulfed the Galactic Republic. The taxation of trade routes to\noutlaying star systems is in dispute.\nHoping to resolve the matter with a blockade of deadly battleships, the\ngreedy Trade Federation has stopped all shipping to the small planet of\nNaboo.\nWhile the congress of the Republic endlessly debates this alarming chain of\nevents, the Supreme Chancellor has secretly dispatched two Jedi Knights,\nthe guardians of peace and justice in the galaxy, to settle the\nconflict.....\n\nPAN DOWN to reveal a small space cruiser heading TOWARD CAMERA at great\nspeed. PAN with the cruiser as it heads towardthe beautiful green planet of\nNaboo, which is surrounded by hundreds of Trade Federation battleships.\n\nINT. REP

In [9]:
import re

word_list = text.split(" ")
cleaned_list = []

for word in word_list:
    cleaned_word = re.sub(r"[^a-zA-Z0-9-_]", " ", word)
    cleaned_list.append(cleaned_word)

cleaned_list = [word.split(" ") for word in cleaned_list]
cleaned_list = [word for sublist in cleaned_list for word in sublist]
cleaned_list = [word.lower() for word in cleaned_list if word != ""]

In [10]:
pattern = r"\bint\b"
scene_numbers = []

for i, word in enumerate(cleaned_list):
    if re.search(pattern, word):
        scene_numbers.append(i)

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

r = requests.get("https://www.imdb.com/title/tt0120915/fullcredits?ref_=tt_ov_st_sm")
soup = bs(r.text, "html.parser")

# character_table = soup.find("div", class_="jhtnKe oJxN6") #.find_all("span", class_="mw-headline")


table = soup.find("table", class_="cast_list")
# character_table = character_table.find_all("div", class_="JjtOHd").text
character_list = table.find_all("td", class_="character")


character_names = []
for character in character_list:
    name = character.find("a").text
    if len(name.split(" ")) <= 3 and name not in character_names:
        character_names.append(name.lower())

In [12]:
characters_dict = {}

for name in character_names:
    if (
        name == "senator palpatine"
        or name == "darth maul"
        or name.startswith("captain")
        or name == "queen amidala"
    ):
        first_name = name.split(" ", 1)[1]
        characters_dict[first_name] = name
    else:
        first_name = name.split(" ", 1)[0]
        characters_dict[first_name] = name

In [13]:
# "jinn", "kenobi", "padmé", "darth sidous"

characters_dict["jinn"] = "qui-gon jinn"
characters_dict["kenobi"] = "obi-wan kenobi"
characters_dict["padmé"] = "queen amidala"
characters_dict["darth sidious"] = "darth sidous"
characters_dict["windu"] = "mace windu"
characters_dict["skywalker"] = "anakin skywalker"
characters_dict["valorum"] = "chancellor valorum"

In [14]:
del characters_dict["coruscant"]
del characters_dict["battle"]
del characters_dict["republic"]
del characters_dict["senate"]
del characters_dict["mawhonic"]
del characters_dict["palace"]
del characters_dict["naboo"]

In [15]:
len(scene_numbers)

92

In [16]:
scenes = []
for i in range(len(scene_numbers) - 1):
    characters_in_scenes = []
    end_i = min(i + 1, scene_numbers[-1])
    for scene_word in cleaned_list[scene_numbers[i] : scene_numbers[end_i]]:
        if scene_word in characters_dict.keys():
            characters_in_scenes.append(characters_dict.get(scene_word))
    scenes.append(characters_in_scenes)

In [17]:
scenes

[['qui-gon jinn',
  'qui-gon jinn',
  'nute gunray',
  'chancellor valorum',
  'nute gunray'],
 ['tc-14', 'tc-14', 'tc-14', 'tc-14'],
 ['tc-14',
  'tc-14',
  'obi-wan kenobi',
  'obi-wan kenobi',
  'qui-gon jinn',
  'qui-gon jinn',
  'obi-wan kenobi',
  'obi-wan kenobi',
  'qui-gon jinn',
  'obi-wan kenobi',
  'qui-gon jinn',
  'obi-wan kenobi',
  'obi-wan kenobi',
  'yoda',
  'qui-gon jinn',
  'obi-wan kenobi',
  'chancellor valorum',
  'qui-gon jinn'],
 ['nute gunray', 'tc-14', 'nute gunray', 'tc-14'],
 ['nute gunray', 'nute gunray', 'tc-14'],
 ['nute gunray',
  'rune haako',
  'rune haako',
  'nute gunray',
  'nute gunray',
  'rune haako',
  'nute gunray',
  'rune haako',
  'nute gunray',
  'rune haako'],
 ['qui-gon jinn', 'obi-wan kenobi', 'qui-gon jinn'],
 ['qui-gon jinn',
  'obi-wan kenobi',
  'nute gunray',
  'qui-gon jinn',
  'rune haako',
  'nute gunray',
  'rune haako'],
 ['qui-gon jinn',
  'qui-gon jinn',
  'obi-wan kenobi',
  'obi-wan kenobi',
  'qui-gon jinn',
  'obi-wan k

In [18]:
relationships = []
for scene in scenes:
    if len(scene) > 1:
        for i, name in enumerate(scene[:-1]):
            pair = scene[i + 1]
            if name != pair:
                relationships.append({"source": name, "target": pair})

In [19]:
network_df = pd.DataFrame(relationships)

In [20]:
import numpy as np

network_df = pd.DataFrame(
    np.sort(network_df.values, axis=1), columns=network_df.columns
)
network_df["weight"] = 1
network_df = network_df.groupby(["source", "target"], sort=False, as_index=False).sum()

In [21]:
network_df = network_df.sort_values(by="weight", ascending=False)

In [22]:
network_df.reset_index(inplace=True, drop=True)

In [23]:
network_df

,source,target,weight
0,obi-wan kenobi,qui-gon jinn,156
1,anakin skywalker,qui-gon jinn,153
2,jar jar binks,qui-gon jinn,115
3,anakin skywalker,shmi skywalker,66
4,anakin skywalker,jar jar binks,58
...,...,...,...
157,jira,watto,1
158,senator palpatine,sio bibble,1
159,gungan guard,senator palpatine,1
160,chancellor valorum,jar jar binks,1


In [24]:
import networkx as nx
import matplotlib.pyplot as plt

In [25]:
G = nx.Graph()

for i, row in network_df.iterrows():
    G.add_edge(row.source, row.target, weight=row.weight)

In [27]:
G = nx.from_pandas_edgelist(
    df=network_df, source="source", target="target", edge_attr="weight"
)
nx.write_gexf(G, "/Users/norat/Desktop/sw1.1.gexf")

In [134]:
text = ""
for i in range(1, 4):
    with open(f"./SW ep{i}.txt", "r") as file:
        text_part = file.read()
        text += text_part

In [135]:
text

'STAR WARS : EPISODE 1:THE PHANTOM MENACE\n\n\n\nTITLE CARD : A long time ago in a galaxy far, far away....\n\nA vast sea of stars serves as the backdrop for the main title, followed by\na roll up, which crawls up into infinity.\n\nEPISODE 1 THE PHANTOM MENACE\n\nTurmoil has engulfed the Galactic Republic. The taxation of trade routes to\noutlaying star systems is in dispute.\nHoping to resolve the matter with a blockade of deadly battleships, the\ngreedy Trade Federation has stopped all shipping to the small planet of\nNaboo.\nWhile the congress of the Republic endlessly debates this alarming chain of\nevents, the Supreme Chancellor has secretly dispatched two Jedi Knights,\nthe guardians of peace and justice in the galaxy, to settle the\nconflict.....\n\nPAN DOWN to reveal a small space cruiser heading TOWARD CAMERA at great\nspeed. PAN with the cruiser as it heads towardthe beautiful green planet of\nNaboo, which is surrounded by hundreds of Trade Federation battleships.\n\nINT. REP

In [136]:
word_list = text.split(" ")
cleaned_list = []

for word in word_list:
    cleaned_word = re.sub(r"[^a-zA-Z0-9-_]", " ", word)
    cleaned_list.append(cleaned_word)

cleaned_list = [word.split(" ") for word in cleaned_list]

cleaned_list = [word for sublist in cleaned_list for word in sublist]

cleaned_list = [word.lower() for word in cleaned_list if word != ""]

In [137]:
urls = [
    "https://www.imdb.com/title/tt0120915/fullcredits?ref_=tt_ov_st_sm",
    "https://www.imdb.com/title/tt0121765/fullcredits?ref_=tt_ov_st_sm",
    "https://www.imdb.com/title/tt0121766/fullcredits?ref_=tt_ov_st_sm",
]

character_names_dict = {}

for url in urls:
    r = requests.get(url)
    soup = bs(r.text, "html.parser")

    # character_table = soup.find("div", class_="jhtnKe oJxN6") #.find_all("span", class_="mw-headline")

    table = soup.find("table", class_="cast_list")
    # character_table = character_table.find_all("div", class_="JjtOHd").text
    character_list = table.find_all("td", class_="character")

    for character in character_list:
        try:
            name = character.find("a").text
            splitted_name = name.split(" ")
            if len(splitted_name) <= 3:
                for i in range(len(splitted_name)):
                    if not splitted_name[i].lower() in character_names_dict.keys():
                        character_names_dict[splitted_name[i].lower()] = name
        except AttributeError as e:
            continue

In [138]:
character_names_dict["sidious"] = "The Emperor"
character_names_dict["palpatine"] = "The Emperor"
character_names_dict["annie"] = "Anakin Skywalker"
character_names_dict["padme"] = "Queen Amidala"
character_names_dict["threepio"] = "C-3PO"
character_names_dict["artoo"] = "R2-D2"

In [139]:
del character_names_dict["of"]
del character_names_dict["the"]
del character_names_dict["republic"]
del character_names_dict["cruiser"]
del character_names_dict["droid"]
del character_names_dict["captain"]
del character_names_dict["jedi"]
del character_names_dict["we"]
del character_names_dict["how"]
del character_names_dict["blue"]
del character_names_dict["senate"]
del character_names_dict["naboo"]

In [140]:
pattern = r"\bint\b"
scene_numbers = []

for i, word in enumerate(cleaned_list):
    if re.search(pattern, word):
        scene_numbers.append(i)

In [141]:
scenes = []
for i in range(len(scene_numbers) - 1):
    characters_in_scenes = []
    end_i = min(i + 1, scene_numbers[-1])
    for scene_word in cleaned_list[scene_numbers[i] : scene_numbers[end_i]]:
        for name in character_names_dict.keys():
            if scene_word == name:
                characters_in_scenes.append(character_names_dict.get(scene_word))
    scenes.append(characters_in_scenes)

In [142]:
scenes

[['Naboo Fighter Pilot',
  'Qui-Gon Jinn',
  'Qui-Gon Jinn',
  'Nute Gunray',
  'Nute Gunray',
  'Pa Due',
  'Supreme Chancellor Palpatine',
  'Chancellor Valorum',
  'Nute Gunray'],
 ['TC-14', 'Runway Worker', 'TC-14', 'TC-14', 'TC-14'],
 ['TC-14',
  'TC-14',
  'Obi-Wan Kenobi',
  'Obi-Wan Kenobi',
  'Qui-Gon Jinn',
  'Qui-Gon Jinn',
  'Obi-Wan Kenobi',
  'Aurra Sing',
  'Obi-Wan Kenobi',
  'Qui-Gon Jinn',
  'Obi-Wan Kenobi',
  'Qui-Gon Jinn',
  'Obi-Wan Kenobi',
  'Obi-Wan Kenobi',
  'Yoda',
  'Qui-Gon Jinn',
  'Obi-Wan Kenobi',
  'Chancellor Valorum',
  'Qui-Gon Jinn'],
 ['Nute Gunray',
  'Nute Gunray',
  'Daultay Dofine',
  'TC-14',
  'Nute Gunray',
  'TC-14',
  'Daultay Dofine'],
 ['Nute Gunray',
  'Battle Droid',
  'Nute Gunray',
  'Battle Droid',
  'Battle Droid',
  'TC-14',
  'Dixon Just',
  'Battle Droid',
  'Fire Speeder Pilot'],
 ['Nute Gunray',
  'Rune Haako',
  'Rune Haako',
  'Nute Gunray',
  'Nute Gunray',
  'Rune Haako',
  'Jedi Knight',
  'Nute Gunray',
  'Rune Haako',

In [143]:
relationships = []
for scene in scenes:
    if len(scene) > 1:
        for i, name in enumerate(scene[:-1]):
            pair = scene[i + 1]
            if name != pair:
                relationships.append({"source": name, "target": pair})

In [144]:
relationships

[{'source': 'Naboo Fighter Pilot', 'target': 'Qui-Gon Jinn'},
 {'source': 'Qui-Gon Jinn', 'target': 'Nute Gunray'},
 {'source': 'Nute Gunray', 'target': 'Pa Due'},
 {'source': 'Pa Due', 'target': 'Supreme Chancellor Palpatine'},
 {'source': 'Supreme Chancellor Palpatine', 'target': 'Chancellor Valorum'},
 {'source': 'Chancellor Valorum', 'target': 'Nute Gunray'},
 {'source': 'TC-14', 'target': 'Runway Worker'},
 {'source': 'Runway Worker', 'target': 'TC-14'},
 {'source': 'TC-14', 'target': 'Obi-Wan Kenobi'},
 {'source': 'Obi-Wan Kenobi', 'target': 'Qui-Gon Jinn'},
 {'source': 'Qui-Gon Jinn', 'target': 'Obi-Wan Kenobi'},
 {'source': 'Obi-Wan Kenobi', 'target': 'Aurra Sing'},
 {'source': 'Aurra Sing', 'target': 'Obi-Wan Kenobi'},
 {'source': 'Obi-Wan Kenobi', 'target': 'Qui-Gon Jinn'},
 {'source': 'Qui-Gon Jinn', 'target': 'Obi-Wan Kenobi'},
 {'source': 'Obi-Wan Kenobi', 'target': 'Qui-Gon Jinn'},
 {'source': 'Qui-Gon Jinn', 'target': 'Obi-Wan Kenobi'},
 {'source': 'Obi-Wan Kenobi', 'tar

In [145]:
network_df = pd.DataFrame(relationships)
network_df = pd.DataFrame(
    np.sort(network_df.values, axis=1), columns=network_df.columns
)
network_df["weight"] = 1
network_df = network_df.groupby(["source", "target"], sort=False, as_index=False).sum()

In [146]:
network_df = network_df.sort_values(by="weight", ascending=False)
network_df.reset_index(inplace=True, drop=True)

In [147]:
network_df

,source,target,weight
0,Anakin Skywalker,Queen Amidala,654
1,Anakin Skywalker,Obi-Wan Kenobi,527
2,Anakin Skywalker,The Emperor,187
3,Obi-Wan Kenobi,Qui-Gon Jinn,146
4,Anakin Skywalker,Qui-Gon Jinn,144
...,...,...,...
1166,Clone War Jedi,Ric Olié,1
1167,Jar Jar Binks,Jedi Knight,1
1168,Jar Jar Binks,Naboo Starship Pilot,1
1169,Count Dooku,Starfreighter Security Officer,1


In [148]:
G = nx.Graph()

for i, row in network_df.iterrows():
    G.add_edge(row.source, row.target, weight=row.weight)

In [149]:
G = nx.from_pandas_edgelist(
    df=network_df, source="source", target="target", edge_attr="weight"
)
nx.write_gexf(G, "/Users/norat/Desktop/sw1_3.gexf")

# Star Wars 4-6

In [150]:
text = ""
for i in range(4, 7):
    with open(f"./SW ep{i}.txt", "r") as file:
        text_part = file.read()
        text += text_part

word_list = text.split(" ")
cleaned_list = []

for word in word_list:
    cleaned_word = re.sub(r"[^a-zA-Z0-9-_]", " ", word)
    cleaned_list.append(cleaned_word)

cleaned_list = [word.split(" ") for word in cleaned_list]

cleaned_list = [word for sublist in cleaned_list for word in sublist]

cleaned_list = [word.lower() for word in cleaned_list if word != ""]

urls = [
    "https://www.imdb.com/title/tt0076759/fullcredits?ref_=tt_ov_st_sm",
    "https://www.imdb.com/title/tt0080684/fullcredits?ref_=tt_ov_st_sm",
    "https://www.imdb.com/title/tt0086190/fullcredits?ref_=tt_ov_st_sm",
]

character_names_dict = {}

for url in urls:
    r = requests.get(url)
    soup = bs(r.text, "html.parser")

    # character_table = soup.find("div", class_="jhtnKe oJxN6") #.find_all("span", class_="mw-headline")

    table = soup.find("table", class_="cast_list")
    # character_table = character_table.find_all("div", class_="JjtOHd").text
    character_list = table.find_all("td", class_="character")

    for character in character_list[:30]:
        try:
            name = character.find("a").text
            splitted_name = name.split(" ")
            if len(splitted_name) <= 3:
                for i in range(len(splitted_name)):
                    if not splitted_name[i].lower() in character_names_dict.keys():
                        character_names_dict[splitted_name[i].lower()] = name
        except AttributeError as e:
            continue

In [151]:
character_names_dict["ben"] = "Obi-Wan Kenobi"
character_names_dict["obi-wan"] = "Obi-Wan Kenobi"
character_names_dict["kenobi"] = "Obi-Wan Kenobi"
character_names_dict["threepio"] = "C-3PO"
character_names_dict["artoo"] = "R2-D2"
character_names_dict["emperor"] = "The Emperor"
character_names_dict["chewie"] = "Chewbacca"

In [152]:
character_names_dict["darth"] = "Anakin Skywalker"
character_names_dict["vader"] = "Anakin Skywalker"

In [153]:
del character_names_dict["the"]

In [154]:
scenes = []
for i in range(len(scene_numbers) - 1):
    characters_in_scenes = []
    end_i = min(i + 1, scene_numbers[-1])
    for scene_word in cleaned_list[scene_numbers[i] : scene_numbers[end_i]]:
        for name in character_names_dict.keys():
            if scene_word == name:
                characters_in_scenes.append(character_names_dict.get(scene_word))
    scenes.append(characters_in_scenes)


relationships = []
for scene in scenes:
    if len(scene) > 1:
        for i, name in enumerate(scene[:-1]):
            pair = scene[i + 1]
            if name != pair:
                relationships.append({"source": name, "target": pair})


network_df_4_6 = pd.DataFrame(relationships)
network_df_4_6 = pd.DataFrame(
    np.sort(network_df_4_6.values, axis=1), columns=network_df_4_6.columns
)
network_df_4_6["weight"] = 1
network_df_4_6 = network_df_4_6.groupby(
    ["source", "target"], sort=False, as_index=False
).sum()
network_df_4_6 = network_df_4_6.sort_values(by="weight", ascending=False)
network_df_4_6.reset_index(inplace=True, drop=True)

In [155]:
network_df_4_6

,source,target,weight
0,Han Solo,Princess Leia Organa,325
1,Han Solo,Luke Skywalker,240
2,Chewbacca,Han Solo,195
3,Anakin Skywalker,Luke Skywalker,188
4,Luke Skywalker,Princess Leia Organa,153
...,...,...,...
333,Snow Creature,The Emperor,1
334,Admiral Piett,Boba Fett,1
335,Boba Fett,Imperial Officer,1
336,Boba Fett,Yoda,1


In [156]:
G = nx.Graph()

In [157]:
G = nx.from_pandas_edgelist(
    df=network_df_4_6, source="source", target="target", edge_attr="weight"
)
nx.write_gexf(G, "/Users/norat/Desktop/sw4_6.gexf")

In [158]:
sw_1_6 = pd.concat([network_df, network_df_4_6], ignore_index=True)

In [159]:
sw_1_6 = sw_1_6.sort_values(by="weight", ascending=False)

In [160]:
sw_1_6.reset_index(inplace=True, drop=True)

In [161]:
G = nx.from_pandas_edgelist(
    df=sw_1_6, source="source", target="target", edge_attr="weight"
)
nx.write_gexf(G, "/Users/norat/Desktop/sw_1_6.gexf")

In [162]:
sw_1_6.to_csv("/Users/norat/Desktop/networkviz/sw1_6.csv")

In [163]:
sw_1_6.loc[sw_1_6.source == "Anakin Skywalker"]

,source,target,weight
0,Anakin Skywalker,Queen Amidala,654
1,Anakin Skywalker,Obi-Wan Kenobi,527
5,Anakin Skywalker,Luke Skywalker,188
6,Anakin Skywalker,The Emperor,187
10,Anakin Skywalker,Qui-Gon Jinn,144
...,...,...,...
1348,Anakin Skywalker,Jobal Naberrie,1
1407,Anakin Skywalker,Aqualish Alian,1
1433,Anakin Skywalker,Coruscant Starfreighter Passenger,1
1476,Anakin Skywalker,Bus Driver,1


In [95]:
G.number_of_edges()

1496

In [200]:
nodes = pd.read_csv("/Users/norat/Desktop/networkviz/nodes.csv")

In [208]:
nodes.loc[nodes.Id == "The Emperor"]

,Id,Label,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality,Weighted Degree,modularity_class
7,The Emperor,The Emperor,3,0.592593,0.663194,874.295453,672,3


In [209]:
labels = nodes.Label.to_list()

In [210]:
degrees = nodes["Weighted Degree"].to_list()

In [211]:
new_labels = []
for idx, degree in enumerate(degrees):
    if degree > 500:
        new_labels.append(labels[idx])
    else:
        new_labels.append("")

In [212]:
nodes_df2 = nodes.copy()

In [213]:
nodes_df2["New labels"] = new_labels

In [214]:
nodes_df2.set_index("Id")[["New labels"]].to_csv("/Users/norat/Desktop/new_labels.csv")